# 13.3 Statsmodels 소개

In [1]:
# statsmodels는 다양한 종류의 통계 모델 피팅, 통계 테스트 수행, 그리고 데이터 탐색과 시각화를 위한 파이썬 라이브러리다.
# statsmodel는 좀 더 "전통적인" 빈도주의적 통계 메서드를 포함하고 있다.
# 베이지안 메서드나 머신러닝 모델은 다른 라이브러리에서 찾을 수 있다.

In [ ]:
# statsmodel는 다음과 같은 모델을 포함한다.

- 선형 모델, 일반 선형 모델, 로버스트 선형 모델
- 선형 복합효과(Linear Mixed Effect, LME) 모델
- 아노바(ANOVA) 메서드
- 시계열 처리 및 상태 공간 모델
- 일반적률추정법(Generalized Method of Moments, GMM)

In [2]:
# 이제 statsmodel의 몇 가지 기본 도구를 사용해보고 Patsy와 pandas의 DataFrame 객체와 모델링 인터페이스를 어떻게 사용하는지 살펴보자.

- 13.3.1 선형 모델 예측하기

In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [4]:
# statsmodels에는 아주 기본적인 선형회귀 모델(예를 들면 최소제곱(OLS))부터 더욱 복잡한 선형회귀 모델(반복재가중 최소제곱(IRLS))까지 존재한다.
# statsmodels의 선형 모델은 두 가지 주요한 인터페이스를 가지는데, 배열 기반의 용법 기반이다. 
# 이 인터페이스는 위처럼 API 모듈을 임포트하여 사용할 수 있다.

In [5]:
# 어떻게 사용하는지 알아보기 위해 랜덤 데이터에서 선형 모델을 생성해보자.

In [10]:
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * np.random.randn(*size)

# 동일한 난수 발생을 위한 seed값 적용
np.random.seed(12345)

N = 100
X = np.c_[dnorm(0, 0.4, size=N),
          dnorm(0, 0.6, size=N),
          dnorm(0, 0.2, size=N)]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

In [11]:
# 여기서는 알려진 인자인 beta를 이용하여 진짜 모델을 작성했다. dnorm은 특정 평균과 분산을 가지는 정규분포 데이터를 생성하기 위한 도움 함수다.
# 이제 다음과 같은 데이터셋을 가지게 되었다.

In [13]:
X[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [15]:
y[:5]

array([ 0.42786349, -0.67348041, -0.09087764, -0.48949442, -0.12894109])

In [16]:
# 선형 모델은 이전에 Patsy에서 봤던 것처럼 일반적으로 intercept와 함께 피팅된다.
# sm.add_constant 함수는 intercept 컬럼을 기존 행렬에 더할 수 있다.

In [17]:
X_model = sm.add_constant(X)

In [18]:
X_model[:5]

array([[ 1.        , -0.12946849, -1.21275292,  0.50422488],
       [ 1.        ,  0.30291036, -0.43574176, -0.25417986],
       [ 1.        , -0.32852189, -0.02530153,  0.13835097],
       [ 1.        , -0.35147471, -0.71960511, -0.25821463],
       [ 1.        ,  1.2432688 , -0.37379916, -0.52262905]])

In [19]:
# sm.OLS 클래스는 최소자승 선형회귀에 피팅할 수 있다.

In [20]:
model = sm.OLS(y, X)

In [21]:
# 모델의 fit 메서드는 예측 모델 인자와 다른 분석 정보를 포함하는 회귀 결과 객체를 반환한다.

In [22]:
results = model.fit()

In [23]:
results.params

array([0.17826108, 0.22303962, 0.50095093])

In [24]:
# results의 summary 메서드를 호출하여 해당 모델의 자세한 분석 결과를 출력하도록 할 수 있다.

In [25]:
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.430
Model:                            OLS   Adj. R-squared (uncentered):              0.413
Method:                 Least Squares   F-statistic:                              24.42
Date:                Mon, 06 Feb 2023   Prob (F-statistic):                    7.44e-12
Time:                        18:45:26   Log-Likelihood:                         -34.305
No. Observations:                 100   AIC:                                      74.61
Df Residuals:                      97   BIC:                                      82.42
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [27]:
# 지금까지는 인자의 이름을 x1, x2 등으로 지었다. 그 대신 모든 모델의 인자가 하나의 DataFrame에 들어있다고 가정해보자.

In [28]:
data = pd.DataFrame(X, columns=["col0", "col1", "col2"])

In [29]:
data["y"] = y

In [30]:
data[:5]

,col0,col1,col2,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941


In [31]:
# 이제 statsmodels의 API와 Patsy의 문자열 용법을 사용할 수 있다.

In [34]:
results = smf.ols("y ~ col0 + col1 + col2", data=data).fit()

In [35]:
results.params

Intercept    0.033559
col0         0.176149
col1         0.224826
col2         0.514808
dtype: float64

In [36]:
results.tvalues

Intercept    0.952188
col0         3.319754
col1         4.850730
col2         6.303971
dtype: float64

In [37]:
# statsmodels에서 반환하는 결과 Series가 DataFrame의 컬럼 이름을 사용하고 있는 것을 알 수 있다. 
# 또한 pandas 객체를 이용해서 수식을 사용하는 경우에는 add_constant를 호출할 필요가 없다.

In [38]:
# 주어진 새로운 샘플 데이터를 통해 예측 모델 인자에 전달한 예측값을 계산할 수 있다.

In [39]:
results.predict(data[:5])

0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

In [40]:
# statsmodels에는 선형 모델 결과에 대한 분석, 진단 그리고 시각화를 위한 많은 추가적인 도구가 포함되어 있다.
# 최소제곱뿐만 아니라 다른 선형 모델에 대한 것들도 포함하고 있다.

- 13.3.2 시계열 처리 예측

In [41]:
# statsmodels에 포함된 또 다른 모델 클래스로는 시계열분석을 위한 모델이 있다.
# 시계열분석을 위한 모델에는 자동회귀 처리, 칼만 필터링과 다른 상태 공간 모델 그리고 다변 자동회귀 모델 등이 있다.

In [42]:
# 자동회귀 구조와 노이즈를 이용해서 시계열 데이터를 시뮬레이션해보자.

In [43]:
init_x = 4

import random
values = [init_x, init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

In [44]:
# 이 데이터는 인자가 0.8과 -0.4인 AR(2) 구조(두 개의 지연)다. 
# AR 모델을 피팅할 때는 포함시켜야 할 지연 항을 얼마나 두어야 하는지 알지 못하므로 적당히 큰 값으로 모델을 피팅한다.

In [47]:
from statsmodels.tsa.ar_model import AutoReg
MAXLAGS = 5
model = AutoReg(values, MAXLAGS)
results = model.fit()

In [48]:
# 결과에서 예측된 인자는 intercept를 가지고 그 다음에 두 지연에 대한 예측치를 갖는다.

In [49]:
results.params

array([-0.00616093,  0.78446347, -0.40847891, -0.01364148,  0.01496872,
        0.01429462])

In [ ]:
# 이런 모델에 대한 심도 있는 내용과 결과를 어떻게 해석해야 하는지는 이 책에서 다루려는 내용이 아니다.